In [2]:
#process the .faa for every AAv serotype
#to know the length of its Rep  and Cap components
#output: virus dictionary: dv ={virus_ref:(virus_seroptype, rep_cap, length)}


import os
vpath = "C:/Users/abidd/Dropbox/algobio/data-aav/" #virus path
dv = {}

for f in os.listdir(vpath): #loop over all the serotypes
    if f.endswith(".faa"):
        aav = f[0:4]
        with open(vpath + f, 'r') as faa: #open every .faa file and loop over its lines
            for line in faa:
                if line.startswith(">"):
                    l = 0 #initialize the length of aa to zero
                    #extract Rep or Cap reference
                    ref = line[line.find("ref")+4: line.find("|", line.find("ref")+4)] 
                    #extract Rep or Cap or others
                    rc = line[line.find("|", line.find("ref")+4)+1: line.find("[", line.find("|", line.find("ref")+4)+1)]
                else:
                    #count number aa in that sequence
                    l += len(line.strip(' \t\n\r\n')) #trim any white space or carriage return
                    dv[ref] = (aav, rc, l)
#print dv

In [3]:
#process the output of the tblastn file (tabular format)/ input: tblastn-o6-100.dat
#the output of this script is of format: /output: tblastn-o6-100-p.dat
#aav,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
#AAV1,gi|9632549|ref|NP_049541.1|,chr4,34.247,73,43,2,387,459,106213428,106213225,1.8,38.1
rpath = "C:/Users/abidd/Dropbox/algobio/res-tblastn/" #"/research/dabid/results/"
ifname = "tblastn-o6-100-B45.dat"
ofname = "tblastn-o6-100-B45-p.csv"

import os
import re

#delete the output file if exist
try:
    os.remove(rpath+ofname)
except OSError:
    print ofname, "output file cannot be deleted"
    pass

#write the header of the output file
with open(rpath + ofname, 'a') as ouf:
    ouf.write("rc,aav,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore\r")  
    #rc refers to rep/cap

#process the output file 
with open(rpath + ifname, 'r') as inf:
    for line in inf:
        if line.find("Adeno") != -1: #serotype row, skip it
            #aav= line[25:len(line)-10]
            continue
        else:
            with open(rpath + ofname, 'a') as ouf: #to write the remaining of the line
                ref= line[line.find("ref")+4: line.find("|", line.find("ref")+4)] 
                aav, rc, l = dv[ref]
                pline = re.sub('\t', ',', line) #subsitute all the tab to ","
                ouf.write(rc.strip(' ')+','+aav.upper()+','+pline)  
        

In [3]:
#%matplotlib notebook #to plot the figure inline on notebook

import matplotlib.pyplot as plt
import plotly.plotly as py
import numpy as np
import matplotlib.cm as cm
import colorsys

data_dict ={}

with open(ipath + ofname, 'r') as inf:
    for line in inf:
        if line.find("aav") != -1:
            continue
        aav,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore = line.split(",")
        if data_dict.get(sseqid) is None:
            data_dict[sseqid] = [float(pident)]
        else:
            data_dict[sseqid] = data_dict[sseqid]+ [float(pident)]

#in case of using customized colors
#number = 10
#cmap = plt.get_cmap('gnuplot')
#colors = [cmap(i) for i in np.linspace(0, 0.9, number)]

fig = plt.figure(figsize=(13, 5), dpi=100) #define the size of the figure

for chrs in data_dict.keys():#go through the data_dict to plot 
#for chrs, color xzip(in data_dict.keys(), colors) #to use the customized colors
    if len(data_dict[chrs])<=9: #skip the chromosome that have less than 10 hits
        print 'chromosome', chrs, ', nbre of hits', len(data_dict[chrs]) # to output all the skipped chromosomes
        continue
    plt.hist(data_dict[chrs], label=chrs, histtype = 'step') #color=color

plt.legend(loc='upper center', ncol=5, mode="expand", borderaxespad=0.)
plt.xlim(20, 45)
plt.ylim(0, 20)
plt.xlabel('Percentage of matches', fontsize=18)
plt.ylabel('Number of mapped matches', fontsize=16)
plt.xticks(np.arange(20, 45, 1))
plt.show()

chromosome chr8_KI270926v1_alt , nbre of hits 1
chromosome chrX , nbre of hits 5
chromosome chr13 , nbre of hits 8
chromosome chr11 , nbre of hits 5
chromosome chr10 , nbre of hits 1
chromosome chr17 , nbre of hits 8
chromosome chr16 , nbre of hits 3
chromosome chr15 , nbre of hits 3
chromosome chr14 , nbre of hits 9
chromosome chr19 , nbre of hits 4
chromosome chr18 , nbre of hits 5
chromosome chr9_KI270823v1_alt , nbre of hits 1
chromosome chr22 , nbre of hits 9
chromosome chr21 , nbre of hits 2
chromosome chr7 , nbre of hits 9
chromosome chr3 , nbre of hits 9
chromosome chr1 , nbre of hits 7
chromosome chr9 , nbre of hits 3
chromosome chr17_KI270857v1_alt , nbre of hits 1
chromosome chr14_KI270846v1_alt , nbre of hits 1
